# Uploading on Google Colab to train model


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
ROOT_DIR = '/content/gdrive/My Drive/DIP1'


In [ ]:
!ls /content/gdrive/My\ Drive/DIP1

# Installing Ultralytics for Yolov8

In [ ]:
!pip install ultralytics

# Training Model

In [ ]:
import os

from ultralytics import YOLO


# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch

# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "colab_config.yaml"), epochs=50, project="/content/runs", batch = 8, lr0=0.001)  # train the model

In [ ]:
import shutil
import os
import time

source = '/content/runs'
destination = '/content/gdrive/My Drive/DIP1/runs/detect'

os.makedirs(destination, exist_ok=True)

try:
    shutil.copytree(source, destination, dirs_exist_ok=True)
    print(f"Directory copied successfully from {source} to {destination}")
except Exception as e:
    print(f"Error occurred while copying: {e}")

time.sleep(10)

# Testing model on a picture

In [ ]:
from ultralytics import YOLO
import cv2

model = YOLO('runs/detect/train(81)/weights/best.pt')  

image_path = os.path.join('data', 'images', 'L0241.png')
img = cv2.imread(image_path)
results = model(img)

annotated_img = results[0].plot()

cv2.imshow("YOLOv8 Detection", annotated_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Installing necessary dependencies

In [ ]:
!pip install opencv-python

In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os

# Installing pygame to play sound when performing real time detections

In [ ]:
!pip install pygame

In [ ]:
#pygame check:
import pygame
import os
import time

pygame.mixer.init()

alert_sound_path = 'alert.mp3'


def play_alert_sound():
    if os.path.exists(alert_sound_path):
        print(f"Playing sound from: {alert_sound_path}")
        pygame.mixer.music.load(alert_sound_path)
        pygame.mixer.music.play()
    else:
        print("Error: Alert sound file not found!")


play_alert_sound()


timeout = 10

start_time = time.time()

while pygame.mixer.music.get_busy() and (time.time() - start_time < timeout):
    time.sleep(0.1)

if time.time() - start_time >= timeout:
    print(f"Timeout reached: {timeout} seconds. Stopping sound.")
    pygame.mixer.music.stop()

print("Test finished.")

# Utilising HaarCascade (For better Face Detection) + Pygame

In [ ]:
import cv2
import torch
import time
from ultralytics import YOLO
import threading
import pygame
import os

# Initialize Pygame mixer for sound playback
pygame.mixer.init()

alert_sound_path = 'alert.mp3'

model = YOLO(r'D:\School Work\College\DIP\Drowsiness\Drowsiness\runs\detect\train(81)\weights\last.pt')

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def play_alert_sound():
    if os.path.exists(alert_sound_path):
        print(f"Playing sound from: {alert_sound_path}")
        pygame.mixer.music.load(alert_sound_path)
        pygame.mixer.music.play()
    else:
        print("Error: Alert sound file not found!")

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 320)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

class VideoStream:
    def __init__(self, src=0):
        self.stream = cv2.VideoCapture(src)
        (self.grabbed, self.frame) = self.stream.read()
        self.stopped = False
        self.lock = threading.Lock()

    def start(self):
        threading.Thread(target=self.update, args=()).start()
        return self

    def update(self):
        while not self.stopped:
            (grabbed, frame) = self.stream.read()
            with self.lock:
                self.grabbed = grabbed
                self.frame = frame

    def read(self):
        with self.lock:
            frame = self.frame.copy()
        return frame

    def stop(self):
        self.stopped = True
        self.stream.release()

vs = VideoStream().start()

while True:
    frame = vs.read()

    if frame is None:
        print("Error: Failed to capture image.")
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]

        face_resized = cv2.resize(face_roi, (320, 320))

        results = model.predict(face_resized, conf=0.7, iou=0.5)

        if results and len(results[0].boxes) > 0:
            print("Face detected! Playing alert sound.")
            sound_thread = threading.Thread(target=play_alert_sound)
            sound_thread.start()

        annotated_face = results[0].plot()
        if annotated_face.shape[2] == 1:
            annotated_face = cv2.cvtColor(annotated_face, cv2.COLOR_GRAY2BGR)

        annotated_face_resized = cv2.resize(annotated_face, (w, h))

        frame[y:y+h, x:x+w] = annotated_face_resized

    cv2.imshow("Face Detection with YOLOv8", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

vs.stop()
cv2.destroyAllWindows()
pygame.mixer.music.stop()